## Create Static Bridge MetaData Table

In [0]:
CREATE OR REFRESH LIVE TABLE 02_silver.bridge_metadata
COMMENT "Static metadata for five major European bridges"
AS
SELECT *
FROM VALUES
  (1, 'Millau Viaduct',        2460,  342, 343, 'Tarn Valley, France',               'Cable-stayed viaduct',        2004),
  (2, 'Vasco da Gama Bridge', 17280,  420, 155, 'Lisbon, Portugal',                  'Box girder bridge',           1998),
  (3, 'Øresund Bridge',        7845,  490, 204, 'Copenhagen–Malmö, Denmark/Sweden',  'Cable-stayed & tunnel',       2000),
  (4, '15 July Martyrs Bridge',1560, 1074, 165, 'Istanbul, Turkey',                  'Suspension bridge',           1973),
  (5, 'Forth Bridge',          2467,  521, 110, 'Firth of Forth, Scotland',           'Cantilever railway bridge',   1890)
AS bridge_metadata(
    bridge_id,
    name,
    length_m,
    main_span_m,
    height_m,
    location,
    type,
    opened_year
);


In [0]:
create or refresh streaming live table 02_silver.bridge_temperature
(
  CONSTRAINT valid_event_time EXPECT (event_time IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_temperature_range EXPECT (temperature BETWEEN -20 AND 60)

)
comment "Temperature Enriched with metadeta"
as
select
  t.device_id as bridge_id,
  m.name,
  m.location,
  cast(t.event_time as timestamp) as event_time,
  t.temperature
from stream(LIVE.01_bronze.bridge_temperature) t
left join 02_silver.bridge_metadata m
on t.device_id = m.bridge_id


In [0]:
create or refresh streaming live table 02_silver.bridge_vibration
(
  constraint valid_event_time expect (event_time is not null) on violation drop row,
  constraint valid_vibration_range expect (vibration between 0 and 0.1)
)
comment "Vibration Enriched with metadeta"
as 
select
  v.device_id as bridge_id,
  m.name,
  m.location,
  cast(v.event_time as timestamp) as event_time,
  v.vibration
from stream(live.01_bronze.bridge_vibration) v
left join 02_silver.bridge_metadata m
on v.device_id = m.bridge_id

In [0]:
create or refresh streaming live table 02_silver.bridge_tilt
(
  constraint valid_event_time expect (event_time is not null) on violation drop row,
  constraint valid_tilt_range expect (tilt_angle between -0.005 and 0.005)
)
comment "Tilt Angle Enriched with metadeta"
as 
select
  t.device_id as bridge_id,
  m.name,
  m.location,
  cast(t.event_time as timestamp) as event_time,
  t.tilt_angle
from stream(live.01_bronze.bridge_tilt) t
left join 02_silver.bridge_metadata m
on t.device_id = m.bridge_id